## Imports

In [ ]:
from scipy.optimize import differential_evolution
from sklearn.metrics import accuracy_score
import tensorflow as tf
import numpy as np
import configs
import models

## Initialize the configs

In [ ]:
run_config = configs.get_model_config(
    dataset_name="cifar10", num_classes=10, image_size=32, path="."
)
dataset_config = run_config["dataset_config"]

## Prepare the test dataset

In [ ]:
batch_size = run_config["batch_size"]
(_, _), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test))
test_ds = test_ds.batch(batch_size).prefetch(tf.data.AUTOTUNE)

## Initialize model and populate checkpoints

Download the checkpoints from here and then unzip:

```shell
$ wget https://github.com/sayakpaul/parameter-ensemble-differential-evolution/releases/download/v0.1.0/run-files.zip
$ unzip -q run-files.zip
```

Or you can train two networks:

```shell
$ for i in `seq 1 2`; python train.py; done
```

In [ ]:
model_ckpt_paths = tf.io.gfile.glob("*-cifar10")
sorted(model_ckpt_paths)

In [ ]:
model_one = models.get_shallow_cnn(run_config)
model_one.load_weights(tf.train.latest_checkpoint(model_ckpt_paths[0]))

model_two = models.get_shallow_cnn(run_config)
model_two.load_weights(tf.train.latest_checkpoint(model_ckpt_paths[1]))

print("")

## Ensemble without differential evolution

In [ ]:
ensembled_model = models.get_shallow_cnn(run_config)
ensembled_model.compile(
    loss="sparse_categorical_crossentropy", metrics=["accuracy"], optimizer="adam"
)
ema = 0.01

for i in range(len(ensembled_model.layers)):
    if hasattr(ensembled_model.layers[i], "kernel") and (
        hasattr(ensembled_model.layers[i], "bias")
    ):
        ensembled_weights = (
            model_one.layers[i].kernel * ema + (1 - ema) * model_two.layers[i].kernel
        )

        ensembled_bias = (
            model_one.layers[i].bias * ema + (1 - ema) * model_two.layers[i].bias
        )

        ensembled_model.layers[i].kernel.assign(tf.Variable(ensembled_weights))
        ensembled_model.layers[i].bias.assign(tf.Variable(ensembled_bias))

In [ ]:
_, accuracy = ensembled_model.evaluate(test_ds)
print(f"Test accuracy: {round(accuracy * 100, 2)}%")

## Ensemble with differential evolution

In [ ]:
def evaluate_model(ema_factor, members, test_ds):
    ensembled_model = models.get_shallow_cnn(run_config)
    ensembled_model.compile(
        loss="sparse_categorical_crossentropy", metrics=["accuracy"], optimizer="adam"
    )

    for i in range(len(ensembled_model.layers)):
        if hasattr(ensembled_model.layers[i], "kernel") and (
            hasattr(ensembled_model.layers[i], "bias")
        ):
            ensembled_weights = (
                members[0].layers[i].kernel * ema
                + (1 - ema) * members[1].layers[i].kernel
            )

            ensembled_bias = (
                members[0].layers[i].bias * ema_factor
                + (1 - ema_factor) * members[1].layers[i].bias
            )

            ensembled_model.layers[i].kernel.assign(tf.Variable(ensembled_weights))
            ensembled_model.layers[i].bias.assign(tf.Variable(ensembled_bias))

    _, accuracy = ensembled_model.evaluate(test_ds, verbose=0)
    return accuracy


def loss_evolution(ema_factor, members, test_ds):
    return 1.0 - evaluate_model(ema_factor, members, test_ds)

In [ ]:
# Reference: https://machinelearningmastery.com/weighted-average-ensemble-for-deep-learning-neural-networks/
members = [model_one, model_two]

search_arg = (members, test_ds)
result = differential_evolution(
    loss_evolution, bounds=[(0, 1)], args=search_arg, maxiter=1000, tol=1e-7,
)

In [ ]:
print(f"EMA factor found using differential evolution: {result['x']}.")
accuracy = evaluate_model(result["x"], members, test_ds)
print(f"Test accuracy: {round(accuracy * 100, 2)}%")

## Scoring a randomly initialized model

In [ ]:
ensembled_model = models.get_shallow_cnn(run_config)
ensembled_model.compile(
    loss="sparse_categorical_crossentropy", metrics=["accuracy"], optimizer="adam"
)

_, accuracy = ensembled_model.evaluate(test_ds)
print(f"Test accuracy: {round(accuracy * 100, 2)}%")

## Scores of the individual models

In [ ]:
for model in members:
    model.compile(
        loss="sparse_categorical_crossentropy", metrics=["accuracy"], optimizer="adam"
    )

    _, accuracy = model.evaluate(test_ds)
    print(f"Test accuracy: {round(accuracy * 100, 2)}%")

## Classic ensembling

In [ ]:
# Reference: https://machinelearningmastery.com/model-averaging-ensemble-for-deep-learning-neural-networks/
def ensemble_predictions(members):
    yhats = [model.predict(test_ds.map(lambda x, y: x)) for model in members]
    yhats = np.array(yhats)
    summed = np.sum(yhats, axis=0)
    result = np.argmax(summed, axis=1)
    return result


result = ensemble_predictions(members)

y_true = []
for _, label in test_ds.unbatch():
    y_true.append(label)

accuracy = accuracy_score(y_true, result)
print(f"Test accuracy: {round(accuracy * 100, 2)}%")